>This notebook is for extracting the airways in a given square region and then transformin the co-ordinates of those airway waypoints to interface co-ordinates. Later in this notebook , we code to generate structured scenarios using "classLib_Yash.py" 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from functools import partial
import pyproj

In [ ]:
Path.cwd()

In [ ]:
os.chdir('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work')
Path.cwd()

In [ ]:
airway = pd.read_json('airways.json')
airway.head(3)

In [ ]:
def filter_airways(x1,y1,x2,y2,x3,y3,x4,y4,airway):
    filtered_airways = pd.DataFrame()

    boundary_coordinates = ([x1,y1],[x2,y2],[x3,y3],[x4,y4])

    poly = Polygon(boundary_coordinates)
    for i in range(len(airway)):
        point1 = Point([float(airway.segments[i][0]['lat']),float(airway.segments[i][0]['lon'])])
        point2 = Point([float(airway.segments[i][1]['lat']),float(airway.segments[i][1]['lon'])])
        if poly.contains(point1)== True and poly.contains(point2)== True:
            filtered_airways=filtered_airways.append(airway.loc[i])
    return filtered_airways

filteraws = filter_airways(1.19,103.06,1.19,104.7,1.76,104.7,1.76,103.06,airway)
filteraws=filteraws.reset_index(drop=True)
filteraws

In [ ]:
filteraws.name[0]

In [ ]:

horizontal_len=1.64
vertical_len = 0.57
a= np.array([1.19,103.06])
a_ = np.array([a[1],a[0]])
b=  np.array([1.19,104.7])
b_ = np.array([b[1],b[0]])
c =np.array([1.76,104.7])
c_ = np.array([c[1],c[0]])
d=  np.array([1.76,103.06])
d_ = np.array([d[1],d[0]])

dict1=[]

for i in range(len(filteraws)):
    print(i)
    d1= []
    for j in range(len(filteraws.segments[i])):
        # print(j)
        P = np.array([float(filteraws.segments[i][j]['lon']),float(filteraws.segments[i][j]['lat'])])
        vert_d = np.linalg.norm(np.cross(b_-a_,P-a_))/np.linalg.norm(b_-a_)
        horiz_d = np.linalg.norm(np.cross(d_-a_,P-a_))/np.linalg.norm(d_-a_)
        airdict1={
                # "name":filteraws.name[i],        
                # "segments":[{"name":filteraws.segments[i][j]['name'],
                "name":filteraws.segments[i][j]['name'],
                "lat":filteraws.segments[i][j]['lat'],
                "lon":filteraws.segments[i][j]['lon'],
                "latdms":filteraws.segments[i][j]['latdms'],
                "londms":filteraws.segments[i][j]['londms'],
                "transf_x": (900/horizontal_len)*horiz_d,
                "transf_y": (900/vertical_len)*vert_d
                }
        d1.append(airdict1)
    # dict2={"segments":d1}
    dict3 ={"name":filteraws.name[i], 
            "segments":d1}
    # print(dict3)
    dict1.append(dict3)
        
print(dict1)
        
        # print(d1)
    # airdict2 = {"name":filteraws.name[i],
                # "segments":airdict1}
    # print(airdict2)
    # d1


In [ ]:
import json
with open('filtered_transformed_airways.json', 'w') as json_file:
    json.dump(dict1, json_file)

## creating structured scenarios after filtering a sqaure region

In [1]:
import numpy as np 
import pandas as pd 
from classLib_yash_Copy1 import *
import random
from pathlib import Path


In [2]:
os.chdir('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work')
Path.cwd()

WindowsPath('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work')

In [3]:
datalist=[]
for scenarioSet in range(10): 
    df = pd.read_json('filtered_transformed_airways.json')
    waypointList=[]
    for i in range(len(df)):
        for j in range(len(df.segments[i])):
            waypointList.append(WayPoint(df.segments[i][j]['name'],[df.segments[i][j]['transf_x'],900-df.segments[i][j]['transf_y']]))

    airwayList=[]
    l1= [i for i in zip(*[iter(waypointList)]*2)]
    for i in range(len(l1)):
        airwayList.append(Airways('aw'+str(i), l1[i][0],l1[i][1]))
    # len(airwayList) ,len(waypointList)

    ac_per_scenario = 8
    aircraftList=[]
    aircraft_name=[]
    for n in range(1,ac_per_scenario+1):
        aircraft_name.append('ac'+str(n))
    for i in range(ac_per_scenario):
        point=np.random.choice(airwayList)
        d=[20,30,40,50,60,80,100,120,140,150,170,190,210,220,230,240,250] #offset,
        aircraft = Aircraft(str(aircraft_name[i]), point , np.random.choice(d)) #this randomization will create multiple scenaios
        aircraftList.append(aircraft)


    d= Scenario(waypointList,airwayList)
    dict4={}
    for i in range(len(waypointList)):
        waydict={waypointList[i].name:{
            'x':waypointList[i].x,
            'y':waypointList[i].y}}
        dict4.update(waydict)

    dict5={}
    for i in range(len(airwayList)):
        airwaydict= {airwayList[i].name:{
            'start':airwayList[i].start_wp,
            'end' : airwayList[i].end_wp}}
        dict5.update(airwaydict)

    dict6={}
    for i in range(len(aircraftList)):
        aircraftdict= {aircraftList[i].name:{
            'x':aircraftList[i].offset_x,
            'y':aircraftList[i].offset_y,
            'dir_x': aircraftList[i].dir_x,
            'dir_y': aircraftList[i].dir_y}}
        dict6.update(aircraftdict)

    data_dict={'waypoints':dict4, 'airways':dict5, 'aircrafts':dict6}
    datalist.append(data_dict)        


In [4]:
import json
with open('structuredScenarios_set3.json', 'w') as json_file:
    json.dump(datalist, json_file)


NameError: name 'pd' is not defined